# Notebook 1 – Exploración de Datos (EDA) con Datos de Accidentes de Tránsito

**Objetivo del notebook:**  
Aprender a realizar una exploración de datos (EDA) básica y limpieza inicial usando un dataset real sobre accidentes de tránsito y seguros en distintos estados de EE. UU.

**Dataset:**  
Cada fila representa un estado y contiene indicadores de choques fatales y costos de seguros:

- `State`
- `Number of drivers involved in fatal collisions per billion miles`
- `Percentage Of Drivers Involved In Fatal Collisions Who Were Speeding`
- `Percentage Of Drivers Involved In Fatal Collisions Who Were Alcohol-Impaired`
- `Percentage Of Drivers Involved In Fatal Collisions Who Were Not Distracted`
- `Percentage Of Drivers Involved In Fatal Collisions Who Had Not Been Involved In Any Previous Accidents`
- `Car Insurance Premiums ($)`
- `Losses incurred by insurance companies for collisions per insured driver ($)`

En este notebook vamos a:

1. Importar el dataset y entender su estructura.
2. Revisar tipos de datos, valores faltantes y duplicados.
3. Detectar outliers numéricos de forma simple.
4. Crear algunas variables nuevas que tengan sentido analítico.
5. Dejar un **dataframe limpio y listo** para usar en modelos en el siguiente notebook.



In [3]:
# ============================================================
# 1. Importar librerías y cargar el dataset
# ============================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Opciones de visualización
pd.set_option('display.max_columns', None)
plt.style.use('default')

# Cargar el dataset (ajusta la ruta al archivo si es necesario)
# Por ejemplo: 'bad-drivers.csv' en la misma carpeta del notebook
df = pd.read_csv("bad-drivers.csv")

# Mirar las primeras filas
df.head()



,State,Number of drivers involved in fatal collisions per billion miles,Percentage Of Drivers Involved In Fatal Collisions Who Were Speeding,Percentage Of Drivers Involved In Fatal Collisions Who Were Alcohol-Impaired,Percentage Of Drivers Involved In Fatal Collisions Who Were Not Distracted,Percentage Of Drivers Involved In Fatal Collisions Who Had Not Been Involved In Any Previous Accidents,Car Insurance Premiums ($),Losses incurred by insurance companies for collisions per insured driver ($)
0,Alabama,18.8,39,30,96,80,784.55,145.08
1,Alaska,18.1,41,25,90,94,1053.48,133.93
2,Arizona,18.6,35,28,84,96,899.47,110.35
3,Arkansas,22.4,18,26,94,95,827.34,142.39
4,California,12.0,35,28,91,89,878.41,165.63


## 2. Inspección inicial del dataset

En esta sección queremos responder rápidamente:

- ¿Cuántas filas y columnas tiene el dataset?
- ¿Qué tipo de dato tiene cada columna?
- ¿Hay valores faltantes?
- ¿Cómo lucen las estadísticas básicas de las columnas numéricas?



In [4]:
# Dimensiones del dataset
print("Dimensiones del dataset (filas, columnas):", df.shape)

# Información general de tipos de datos y valores no nulos
df.info()


Dimensiones del dataset (filas, columnas): (51, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 8 columns):
 #   Column                                                                                                  Non-Null Count  Dtype  
---  ------                                                                                                  --------------  -----  
 0   State                                                                                                   51 non-null     object 
 1   Number of drivers involved in fatal collisions per billion miles                                        51 non-null     float64
 2   Percentage Of Drivers Involved In Fatal Collisions Who Were Speeding                                    51 non-null     int64  
 3   Percentage Of Drivers Involved In Fatal Collisions Who Were Alcohol-Impaired                            51 non-null     int64  
 4   Percentage Of Drivers Involved In Fatal Coll

In [5]:
# Estadísticas descriptivas de las columnas numéricas
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Number of drivers involved in fatal collisions per billion miles,51.0,15.790196,4.122002,5.90,12.750,15.60,18.500,23.90
Percentage Of Drivers Involved In Fatal Collisions Who Were Speeding,51.0,31.725490,9.633438,13.00,23.000,34.00,38.000,54.00
Percentage Of Drivers Involved In Fatal Collisions Who Were Alcohol-Impaired,51.0,30.686275,5.132213,16.00,28.000,30.00,33.000,44.00
Percentage Of Drivers Involved In Fatal Collisions Who Were Not Distracted,51.0,85.921569,15.158949,10.00,83.000,88.00,95.000,100.00
Percentage Of Drivers Involved In Fatal Collisions Who Had Not Been Involved In Any Previous Accidents,51.0,88.725490,6.960110,76.00,83.500,88.00,95.000,100.00
Car Insurance Premiums ($),51.0,886.957647,178.296285,641.96,768.430,858.97,1007.945,1301.52
Losses incurred by insurance companies for collisions per insured driver ($),51.0,134.493137,24.835922,82.75,114.645,136.05,151.870,194.78


### Preguntas rápidas (para discutir en voz alta / comentar en celdas Markdown):

1. ¿Te llama la atención algún rango de valores (mínimo, máximo, promedio) en las variables numéricas?
2. ¿Hay columnas con valores muy diferentes entre el mínimo y el máximo que podrían indicar presencia de **outliers**?
3. ¿Qué columnas parecen claramente numéricas y cuál es la única claramente categórica?


## 3. Renombrar columnas a un formato más manejable

Los nombres originales son largos. Vamos a renombrarlos a `snake_case` para facilitar el trabajo en Python.

Además, esto es una **buena práctica** para futuros modelos.


In [6]:
# Mostrar nombres de columnas originales
df.columns.tolist()


['State',
 'Number of drivers involved in fatal collisions per billion miles',
 'Percentage Of Drivers Involved In Fatal Collisions Who Were Speeding',
 'Percentage Of Drivers Involved In Fatal Collisions Who Were Alcohol-Impaired',
 'Percentage Of Drivers Involved In Fatal Collisions Who Were Not Distracted',
 'Percentage Of Drivers Involved In Fatal Collisions Who Had Not Been Involved In Any Previous Accidents',
 'Car Insurance Premiums ($)',
 'Losses incurred by insurance companies for collisions per insured driver ($)']

In [7]:
# Diccionario de renombre de columnas
rename_dict = {
    "State": "state",
    "Number of drivers involved in fatal collisions per billion miles": "drivers_fatal_per_billion_miles",
    "Percentage Of Drivers Involved In Fatal Collisions Who Were Speeding": "pct_speeding",
    "Percentage Of Drivers Involved In Fatal Collisions Who Were Alcohol-Impaired": "pct_alcohol",
    "Percentage Of Drivers Involved In Fatal Collisions Who Were Not Distracted": "pct_not_distracted",
    "Percentage Of Drivers Involved In Fatal Collisions Who Had Not Been Involved In Any Previous Accidents": "pct_no_previous_accidents",
    "Car Insurance Premiums ($)": "car_insurance_premiums",
    "Losses incurred by insurance companies for collisions per insured driver ($)": "losses_per_insured_driver",
}

df = df.rename(columns=rename_dict)

df.head()


,state,drivers_fatal_per_billion_miles,pct_speeding,pct_alcohol,pct_not_distracted,pct_no_previous_accidents,car_insurance_premiums,losses_per_insured_driver
0,Alabama,18.8,39,30,96,80,784.55,145.08
1,Alaska,18.1,41,25,90,94,1053.48,133.93
2,Arizona,18.6,35,28,84,96,899.47,110.35
3,Arkansas,22.4,18,26,94,95,827.34,142.39
4,California,12.0,35,28,91,89,878.41,165.63


## 4. Tipos de datos, valores faltantes y duplicados

Antes de hacer modelos o gráficos, debemos asegurarnos de que:

- Cada columna tenga el **tipo de dato correcto** (numérico vs categórico).
- No haya **duplicados** problemáticos.
- Conozcamos la magnitud de los **valores faltantes**.


In [8]:
# 4.1 Revisar tipos de datos
df.dtypes

state                               object
drivers_fatal_per_billion_miles    float64
pct_speeding                         int64
pct_alcohol                          int64
pct_not_distracted                   int64
pct_no_previous_accidents            int64
car_insurance_premiums             float64
losses_per_insured_driver          float64
dtype: object

In [9]:
# Convertir `state` en tipo categórico (opcional pero recomendable)
df["state"] = df["state"].astype("category")

# Verificamos de nuevo
df.dtypes


state                              category
drivers_fatal_per_billion_miles     float64
pct_speeding                          int64
pct_alcohol                           int64
pct_not_distracted                    int64
pct_no_previous_accidents             int64
car_insurance_premiums              float64
losses_per_insured_driver           float64
dtype: object

In [10]:
# 4.2 Valores faltantes (conteo y porcentaje)
missing_count = df.isna().sum()
missing_pct = (missing_count / len(df) * 100).round(2)

missing_summary = pd.DataFrame({
    "missing_count": missing_count,
    "missing_pct": missing_pct
}).sort_values(by="missing_pct", ascending=False)

missing_summary


,missing_count,missing_pct
state,0,0.0
drivers_fatal_per_billion_miles,0,0.0
pct_speeding,0,0.0
pct_alcohol,0,0.0
pct_not_distracted,0,0.0
pct_no_previous_accidents,0,0.0
car_insurance_premiums,0,0.0
losses_per_insured_driver,0,0.0


In [11]:
# 4.3 Filas duplicadas
num_duplicates = df.duplicated().sum()
print(f"Número de filas duplicadas en el dataset: {num_duplicates}")


Número de filas duplicadas en el dataset: 0


### EJERCICIO 1 (para que programen ustedes)

1. Si existieran filas duplicadas, decidan si corresponde eliminarlas o no.  
2. Si existieran valores faltantes, definan una estrategia simple (por ejemplo, eliminar filas con NA en variables críticas).

> **Sugerencia**: usen `df.drop_duplicates()` y/o `df.dropna()` o imputación sencilla (media/mediana) si tiene sentido.


In [12]:
# EJERCICIO 1: completa la limpieza según las decisiones de tu grupo.
# A modo de ejemplo, aquí dejamos líneas comentadas:

# 1) Eliminar filas completamente duplicadas
# df = df.drop_duplicates()

# 2) Si quisieran eliminar filas con NA en alguna columna específica:
# cols_criticas = ["drivers_fatal_per_billion_miles", "pct_speeding", "pct_alcohol"]
# df = df.dropna(subset=cols_criticas)

# Muestra la nueva forma del dataframe
df.shape



(51, 8)

## 5. Detección simple de outliers (z-score)

Ahora vamos a detectar **outliers** numéricos usando un criterio sencillo:

- Para cada columna numérica, calculamos el **z-score**.
- Marcamos como outlier aquellos registros con |z| > 3 en al menos una columna.

Esto no es perfecto, pero es un buen primer filtro.

In [13]:
from scipy import stats

# Seleccionar solo columnas numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols


['drivers_fatal_per_billion_miles',
 'pct_speeding',
 'pct_alcohol',
 'pct_not_distracted',
 'pct_no_previous_accidents',
 'car_insurance_premiums',
 'losses_per_insured_driver']

In [14]:
# Calcular z-scores para cada columna numérica (ignorando filas con NA)
z_scores = np.abs(stats.zscore(df[numeric_cols], nan_policy='omit'))

# z_scores es un array; construimos una máscara de outliers
outlier_mask = (z_scores > 3).any(axis=1)

print(f"Número de filas marcadas como outlier (|z|>3 en alguna variable numérica): {outlier_mask.sum()}")

# Mostrar algunas filas outlier para inspección
df[outlier_mask].head()


Número de filas marcadas como outlier (|z|>3 en alguna variable numérica): 2


,state,drivers_fatal_per_billion_miles,pct_speeding,pct_alcohol,pct_not_distracted,pct_no_previous_accidents,car_insurance_premiums,losses_per_insured_driver
24,Mississippi,17.6,15,31,10,100,896.07,155.77
49,Wisconsin,13.8,36,33,39,84,670.31,106.62


In [15]:
# EJERCICIO 2: decide qué hacer con los outliers.

# Ejemplo: eliminar outliers (si el grupo decide hacerlo)
# df_clean = df[~outlier_mask].copy()

# Ejemplo alternativo: mantenerlos, pero copiamos de todos modos
df_clean = df.copy()

df_clean.shape

(51, 8)

## 6. Creación de variables nuevas

Vamos a crear algunas variables derivadas que pueden ser útiles luego para modelos:

1. **ratio_losses_premiums**: qué proporción de las primas de seguro se “pierde” en siniestros.  
   \[
   \text{ratio\_losses\_premiums} = \frac{\text{losses\_per\_insured\_driver}}{\text{car\_insurance\_premiums}}
   \]

2. **risk_speed_alcohol**: indicador simple de riesgo combinando % de choques con exceso de velocidad y % con alcohol.  
   \[
   \text{risk\_speed\_alcohol} = \text{pct\_speeding} + \text{pct\_alcohol}
   \]

3. **pct_safe_behaviour**: porcentaje aproximado de conductores que en choques fatales *no estaban distraídos* y *no tenían accidentes previos*.  
   (Esto es una aproximación solo para fines didácticos.)


In [16]:
# 6.1 Crear ratio de pérdidas sobre primas
df_clean["ratio_losses_premiums"] = (
    df_clean["losses_per_insured_driver"] / df_clean["car_insurance_premiums"]
)

# 6.2 Crear indicador combinado de riesgo (velocidad + alcohol)
df_clean["risk_speed_alcohol"] = (
    df_clean["pct_speeding"] + df_clean["pct_alcohol"]
)

# 6.3 Aproximación de "comportamiento seguro" combinando no distraídos y sin accidentes previos
# Ojo: esto no es una fórmula oficial, solo una aproximación pedagógica
df_clean["pct_safe_behaviour"] = (
    df_clean["pct_not_distracted"] * df_clean["pct_no_previous_accidents"] / 100.0
)

df_clean[[
    "state",
    "car_insurance_premiums",
    "losses_per_insured_driver",
    "ratio_losses_premiums",
    "risk_speed_alcohol",
    "pct_safe_behaviour"
]].head()


,state,car_insurance_premiums,losses_per_insured_driver,ratio_losses_premiums,risk_speed_alcohol,pct_safe_behaviour
0,Alabama,784.55,145.08,0.184921,69,76.80
1,Alaska,1053.48,133.93,0.127131,66,84.60
2,Arizona,899.47,110.35,0.122683,63,80.64
3,Arkansas,827.34,142.39,0.172106,44,89.30
4,California,878.41,165.63,0.188557,63,80.99


### EJERCICIO 3

En tu grupo:

1. Propongan **al menos una nueva variable** que tenga sentido para el negocio de seguros / seguridad vial.  
   - Ejemplo: `net_margin_proxy = car_insurance_premiums - losses_per_insured_driver`.  
2. Implementen la columna en el dataframe `df_clean`.  
3. Expliquen en una celda Markdown por qué creen que esa variable podría ser útil en un modelo de predicción de pérdidas o primas.


In [17]:
#EJERCICIO 3: crear tu propia variable

# Ejemplo comentado:
# df_clean["net_margin_proxy"] = (
#     df_clean["car_insurance_premiums"] - df_clean["losses_per_insured_driver"]
# )

# TODO: crea aquí tu variable propuesta
# df_clean["<nombre_de_tu_variable>"] = ...


## 7. Selección de columnas para modelado

El objetivo del siguiente notebook será construir un **modelo de regresión lineal**.  
Una opción razonable es predecir:

- **Target (Y):** `losses_per_insured_driver`  
- **Features (X):** variables que describen el riesgo de choques y las primas.

Ejemplo de columnas para el modelo:

- `drivers_fatal_per_billion_miles`
- `pct_speeding`
- `pct_alcohol`
- `pct_not_distracted`
- `pct_no_previous_accidents`
- `car_insurance_premiums`
- `ratio_losses_premiums`
- `risk_speed_alcohol`
- (y la variable nueva que ustedes definan)


In [18]:
# Definir columnas numéricas candidatas para modelar
feature_cols = [
    "drivers_fatal_per_billion_miles",
    "pct_speeding",
    "pct_alcohol",
    "pct_not_distracted",
    "pct_no_previous_accidents",
    "car_insurance_premiums",
    "ratio_losses_premiums",
    "risk_speed_alcohol",
    # Agregar aquí su variable creada si existe
    # "<nombre_de_tu_variable>",
]

target_col = "losses_per_insured_driver"

# Crear dataframe final para modelado
model_df = df_clean[feature_cols + [target_col]].copy()

# Eliminar filas con NA en estas columnas (si quedara alguna)
model_df = model_df.dropna()

model_df.head()


,drivers_fatal_per_billion_miles,pct_speeding,pct_alcohol,pct_not_distracted,pct_no_previous_accidents,car_insurance_premiums,ratio_losses_premiums,risk_speed_alcohol,losses_per_insured_driver
0,18.8,39,30,96,80,784.55,0.184921,69,145.08
1,18.1,41,25,90,94,1053.48,0.127131,66,133.93
2,18.6,35,28,84,96,899.47,0.122683,63,110.35
3,22.4,18,26,94,95,827.34,0.172106,44,142.39
4,12.0,35,28,91,89,878.41,0.188557,63,165.63


## 8. Checkpoint final del Notebook 1

Al terminar este notebook deberías tener:

- Un dataset con nombres de columnas limpios (`snake_case`).
- Tipos de datos adecuados (estado como categoría, resto numérico).
- Revisión (y decisión) sobre duplicados, valores faltantes y outliers.
- Nuevas variables derivadas con sentido de negocio.
- Un dataframe `model_df` listo para usar en el **Notebook 2 – Regresión Lineal**.

### Preguntas de reflexión (para 5 minutos finales)

1. ¿Qué variable crees que tendrá mayor impacto en las pérdidas de las aseguradoras (`losses_per_insured_driver`) y por qué?  
2. ¿Qué decisiones de limpieza podrían cambiar fuertemente los resultados de un futuro modelo?  
3. Si tuvieras que explicarle a alguien de negocio lo que hiciste en este notebook, ¿cómo lo resumirías en 3 frases?

En la próxima sesión vamos a:

- Dividir `model_df` en conjuntos de entrenamiento y prueba.
- Entrenar un modelo de **regresión lineal**.
- Evaluar métricas (MAE, RMSE, R²).
- Analizar la importancia de las variables.

Guarda este notebook: lo vas a reutilizar directamente.
